In [1]:
import pyodbc, psycopg2

conn = psycopg2.connect(database="depdatabase", user="postgres", password="DEPgroep1", host="vichogent.be", port="40033")

cursor = conn.cursor()

cursor.execute(f"""
select b.balanstotaal, b.net_add_val, b.omzet, k.personeelsbestanden, l.urban
from "KMO" k
JOIN "Locatie" l on k.ondernemingsnummer = l.ondernemingsnummer
JOIN "Balans" b on k.ondernemingsnummer = b.ondernemingsnummer
WHERE b.balanstotaal is not null and b.net_add_val > 0
limit 1000
""")

rows_id_exist = cursor.fetchall()

cursor.close()

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MinMaxScalerExample").getOrCreate()

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "postgresql-42.5.0.jar") \
    .getOrCreate()


df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://vichogent.be:40033/depdatabase") \
    .option("dbtable", "KMO") \
    .option("user", "postgres") \
    .option("password", "DEPdatabase1") \
    .option("driver", "org.postgresql.Driver") \
    .load()


Py4JJavaError: An error occurred while calling o118.load.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.d

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer=StringIndexer(inputCol='initial_time_seconds',outputCol='kaggle_cat')
indexed=indexer.fit(df).transform(df)

for item in indexed.head():
    print(item)